In [70]:
using AxisArrays
using LIBSVM
using MLDataPattern
using Plots
gr()

INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /Users/rdeits/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.


Plots.GRBackend()

In [74]:
include("main.jl")

c

In [75]:
double_integrator = c.MPC.discretize(c.MPC.CTLinearSytstem([0. 1; 0 0], [0. 1]'), 0.1)

c.MPC.DTLinearSystem{Float64}([1.0 0.1; 0.0 1.0],[0.005; 0.1],0.1)

In [76]:
time = Axis{:time}(linspace(0, 9 * double_integrator.Δt, 10))
side = Axis{:side}([:left, :right])
model = c.MPC.create_model(double_integrator, time, side);

In [39]:
results = c.collect_data(model, 1000);
examples = convert(Vector{c.Example}, results);
labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [40]:
@time model = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(model, data_test)

  1.413055 seconds (1.27 M allocations: 53.117 MB, 1.24% gc time)
  0.248212 seconds (146.53 k allocations: 8.264 MB)


([1,2,3,2,2,2,4,3,2,2  …  1,1,3,3,3,1,1,3,2,2],
[0.0274118 0.0322017 … 0.0153858 0.0173512; 0.0118779 0.0255339 … 0.0404374 0.0228165; … ; 0.0516423 0.0205577 … 0.0345833 0.121618; 0.0595353 0.0432153 … 0.0422009 0.0192561])

In [41]:
mean(predicted_labels .== labels_test)

0.4810379241516966

In [83]:
model = c.MPC.dummy_model(time, side);
results = c.collect_data(model, 100);
examples = convert(Vector{c.Example}, results);
labels = [e.best_action for e in examples]
data = hcat([vcat(e.contact, e.duals) for e in examples]...);
(data_train, labels_train), (data_test, labels_test) = splitobs((data, labels), 0.5);

In [84]:
@time model = svmtrain(data_train, labels_train; cost=100., probability=true)
@time predicted_labels, decision_values = svmpredict(model, data_test)
mean(predicted_labels .== labels_test)

  0.004175 seconds (73 allocations: 27.484 KB)
  0.000553 seconds (22 allocations: 39.453 KB)


0.8823529411764706

In [85]:
histogram(labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [86]:
histogram(predicted_labels)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 15 
 
 
 20 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1